Optimize_BINARY

In [18]:
import os
import json
import time
import numpy as np
from z3 import *
from itertools import combinations
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# Helper functions
def parseInstance(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()
        
        # Read m and n
        m = int(lines[0].strip())
        n = int(lines[1].strip())
        
        # Read l1 to lm
        l_values = list(map(int, lines[2].strip().split()))
        
        # Read s1 to sn
        s_values = list(map(int, lines[3].strip().split()))
        
        # Read the distance matrix
        distance_matrix = []
        for line in lines[4:]:
            row = list(map(int, line.strip().split()))
            distance_matrix.append(row)
        
        return {'m': m, 'n': n, 'l_values': l_values, 's_values': s_values, 'distance_matrix': distance_matrix}

def computeBounds(D, m, n):
    # Lower Bound: Minimum distance from the origin to the farthest point
    LB = max(D[0][:n])

    # Upper Bound: Sum of all distances (a very loose upper bound)
    UB = sum([sum(row) for row in D])

    return LB, UB

In [19]:
# SMT Solver Helper Functions
counter = 0
def generate_unique_bool_name():
    global counter
    counter += 1
    return f"unique_bool_{counter}"

def at_most_one_T(bools):
    if len(bools) <= 4:  # base case
        return And([Not(And(b1, b2)) for b1, b2 in combinations(bools, 2)])
    
    # recursive step
    y = Bool(generate_unique_bool_name())
    first = bools[:3]
    first.append(y)
    c_first = at_most_one_T(first)

    last = bools[3:]
    last.insert(0, Not(y))
    c_last = at_most_one_T(last)

    return And(c_first, c_last)

def exactly_one_T(bools):
    return And(at_most_one_T(bools), Or(bools))

In [34]:
def solve_mcp(m, n, l_values, s_values, D, LB, UB, time_limit=300):
    # Decision variables
    X = [[[Bool(f'X_{k}_{i}_{j}') for j in range(n + 1)] for i in range(n + 1)] for k in range(m)]
    Y = [[Bool(f'Y_{k}_{i}') for i in range(n)] for k in range(m)]
    U = [Int(f'U_{i}') for i in range(n)]
    C = [Int(f'C_{k}') for k in range(m)]
    MaxCost = Int('MaxCost')

    # Constraints
    constraints = []

    # Domains for U
    for i in range(n):
        constraints.append(U[i] > 0)
        constraints.append(U[i] <= n)

    # C1: Each item must be delivered by exactly one courier
    for i in range(n):
        constraints.append(Or([X[k][i][j] for j in range(n + 1) if i != j for k in range(m)]))
        constraints.append(Or([X[k][j][i] for j in range(n + 1) if i != j for k in range(m)]))
        for k in range(m):
            constraints.append(at_most_one_T([X[k][j][i] for j in range(n + 1) if i != j]))

    # C2: Item assignment constraints
    for i in range(n):
        for k in range(m):
            constraints.append(Y[k][i] == Or([X[k][i][j] for j in range(n + 1) if i != j]))
            constraints.append(Y[k][i] == Or([X[k][j][i] for j in range(n + 1) if i != j]))
    for i in range(n):
        constraints.append(exactly_one_T([Y[k][i] for k in range(m)]))

    # C3: Load constraints
    for k in range(m):
        constraints.append(Sum([If(Y[k][i], s_values[i], 0) for i in range(n)]) <= l_values[k])

    # C4: Ensure couriers start and end at the origin
    for k in range(m):
        constraints.append(exactly_one_T([X[k][n][j] for j in range(n)]))
        constraints.append(exactly_one_T([X[k][i][n] for i in range(n)]))
        constraints.append(Or([Y[k][i] for i in range(n)]))

    # C5: Subtour elimination constraints
    for i in range(n):
        for j in range(n):
            if i != j:
                arc_traversed = Or([X[k][i][j] for k in range(m)])
                constraints.append(Implies(arc_traversed, U[j] > U[i]))

    # Cost constraints
    for k in range(m):
        constraints.append(C[k] == Sum([If(X[k][i][j], D[i][j], 0) for i in range(n + 1) for j in range(n + 1) if i != j]))
        constraints.append(MaxCost >= C[k])

    constraints.append(Or([MaxCost == C[k] for k in range(m)]))
    constraints.append(MaxCost <= UB)
    constraints.append(MaxCost >= LB)

    # Objective: Minimize the maximum distance traveled by any courier
    solver = Optimize()
    solver.add(constraints)
    solver.minimize(MaxCost)

    start_time = time.time()
    best_solution = None
    best_max_cost = UB

    # Incremental solving with smaller time chunks
    while UB > LB and time.time() - start_time < time_limit:
        mid = (UB + LB) // 2
        solver.push()
        solver.add(MaxCost <= mid)
        solver.set('timeout', int((time_limit - (time.time() - start_time)) * 1000))  # Set timeout for remaining time
        if solver.check() == sat:
            model = solver.model()
            best_solution = {
                'max_distance': model.evaluate(MaxCost).as_long(),
                'assignments': [(k, i + 1) for k in range(m) for i in range(n) if is_true(model.evaluate(Y[k][i]))],
                'routes': [
                    [(i, j) for i in range(n + 1) for j in range(n + 1) if i != j and is_true(model.evaluate(X[k][i][j]))]
                    for k in range(m)
                ]
            }
            best_max_cost = model.evaluate(MaxCost).as_long()
            UB = best_max_cost
        else:
            LB = mid + 1
        solver.pop()

    elapsed_time = time.time() - start_time
    return best_solution, elapsed_time

In [35]:
def generate_coordinates(n):
    # Create a circular layout for the distribution points
    theta = np.linspace(0, 2 * np.pi, n, endpoint=False)
    radius = 1
    coordinates = {i: (radius * np.cos(angle), radius * np.sin(angle)) for i, angle in enumerate(theta)}
    coordinates[n] = (0, 0)  # Origin at the center
    return coordinates

In [36]:
def plot_routes(routes, instance, instance_name, method_name):
    n = instance['n']
    colors = ['r', 'g', 'b', 'c', 'm', 'y']

    coordinates = generate_coordinates(n)

    plt.figure(figsize=(8, 8))
    plt.grid(True)

    # Plot the nodes
    for i in range(n):
        plt.plot(*coordinates[i], 'o', color='white', markersize=10, markeredgecolor='black')
        plt.text(coordinates[i][0], coordinates[i][1], f'd{i+1}', fontsize=12, ha='right')

    # Plot the origin
    plt.plot(*coordinates[n], 'o', color='black', markersize=10)
    plt.text(coordinates[n][0], coordinates[n][1], 'origin', fontsize=12, ha='right')

    # Plot the routes
    for i, route in enumerate(routes):
        color = colors[i % len(colors)]
        for j in range(len(route) - 1):
            start = coordinates[route[j]]
            end = coordinates[route[j+1]]
            plt.arrow(start[0], start[1], end[0] - start[0], end[1] - start[1],
                      color=color, head_width=0.05, length_includes_head=True)

    plt.xlabel('x')
    plt.ylabel('y')

    # Create custom legend
    legend_elements = [Line2D([0], [0], color=colors[i % len(colors)], lw=2, label=f'Courier {i+1}')
                       for i in range(len(routes))]
    plt.legend(handles=legend_elements, loc='upper left')

    # Save plot
    if not os.path.exists('res/SMT/Optimize_Binary/route_map'):
        os.makedirs('res/SMT/Optimize_Binary/route_map')
    plt.savefig(f'res/SMT/Optimize_Binary/route_map/{instance_name}_{method_name}_route.png')
    plt.close()

In [37]:
def save_route_json(instance_name, method_name, time_taken, optimal, obj, sol):
    if not os.path.exists('res/SMT/Optimize_Binary/route'):
        os.makedirs('res/SMT/Optimize_Binary/route')

    route_data = {
        method_name: {
            "time": time_taken,
            "optimal": optimal,
            "obj": obj,
            "sol": sol
        }
    }

    json_path = f'res/SMT/Optimize_Binary/route/{instance_name}_route.json'
    if os.path.exists(json_path):
        with open(json_path, 'r') as f:
            data = json.load(f)
    else:
        data = {}

    data[method_name] = route_data[method_name]

    with open(json_path, 'w') as f:
        json.dump(data, f, indent=4)

In [38]:
def print_solution(instance_name, solution, instance, method_name, time_taken):
    if solution:
        print(f"Instance: {instance_name}")
        print(f"Max Distance: {solution['max_distance']}")
        all_routes = []
        sol = []
        for k in range(len(solution['routes'])):
            items = [i for c, i in solution['assignments'] if c == k]
            route = [instance['n']]  # Start from the origin
            while True:
                current = route[-1]
                next_dest = next((j for i, j in solution['routes'][k] if i == current), instance['n'])
                if next_dest == instance['n']:
                    break
                route.append(next_dest)
            route.append(instance['n'])  # End at the origin
            print(f"Courier {k + 1}: Items {items}")
            print(f"Courier {k + 1} Route: {route}")
            all_routes.append(route)
            sol.append(items)
        plot_routes(all_routes, instance, instance_name, method_name)
        save_route_json(instance_name, method_name, min(time_taken, 300), time_taken <= 300, solution['max_distance'], sol)
    else:
        print(f"Instance: {instance_name}")
        print("No solution found within the time limit")
        save_route_json(instance_name, method_name, 300, False, None, [])

In [39]:
folder_path = 'Instances'
method_name = 'Optimize_Binary'

def read_dat_files(folder_path):
    dat_files = [f for f in os.listdir(folder_path) if f.endswith('.dat')]
    instances = {}

    for dat_file in dat_files:
        file_path = os.path.join(folder_path, dat_file)
        instances[dat_file] = parseInstance(file_path)

    return instances

instances = read_dat_files(folder_path)

for instance_name, instance in instances.items():
    m = instance['m']
    n = instance['n']
    l_values = instance['l_values']
    s_values = instance['s_values']
    D = instance['distance_matrix']
    LB, UB = computeBounds(D, m, n)
    solution, time_taken = solve_mcp(m, n, l_values, s_values, D, LB, UB, time_limit=300)
    print_solution(instance_name, solution, instance, method_name, time_taken)


Instance: inst01.dat
Max Distance: 14
Courier 1: Items [1, 3, 4]
Courier 1 Route: [6, 3, 2, 0, 6]
Courier 2: Items [2, 5, 6]
Courier 2 Route: [6, 1, 4, 5, 6]
Instance: inst02.dat
Max Distance: 226
Courier 1: Items [5, 9]
Courier 1 Route: [9, 8, 4, 9]
Courier 2: Items [4]
Courier 2 Route: [9, 3, 9]
Courier 3: Items [6, 8]
Courier 3 Route: [9, 5, 7, 9]
Courier 4: Items [1]
Courier 4 Route: [9, 0, 9]
Courier 5: Items [2, 7]
Courier 5 Route: [9, 1, 6, 9]
Courier 6: Items [3]
Courier 6 Route: [9, 2, 9]
Instance: inst03.dat
Max Distance: 12
Courier 1: Items [2, 4, 5]
Courier 1 Route: [7, 4, 3, 1, 7]
Courier 2: Items [3, 6]
Courier 2 Route: [7, 5, 2, 7]
Courier 3: Items [1, 7]
Courier 3 Route: [7, 6, 0, 7]
Instance: inst04.dat
Max Distance: 220
Courier 1: Items [8]
Courier 1 Route: [10, 7, 10]
Courier 2: Items [3]
Courier 2 Route: [10, 2, 10]
Courier 3: Items [1]
Courier 3 Route: [10, 0, 10]
Courier 4: Items [5]
Courier 4 Route: [10, 4, 10]
Courier 5: Items [6]
Courier 5 Route: [10, 5, 10]
Co

Solver_Binary

In [40]:
def solve_mcp(m, n, l_values, s_values, D, LB, UB, time_limit=300):
    # Decision variables
    X = [[[Bool(f'X_{k}_{i}_{j}') for j in range(n + 1)] for i in range(n + 1)] for k in range(m)]
    Y = [[Bool(f'Y_{k}_{i}') for i in range(n)] for k in range(m)]
    U = [Int(f'U_{i}') for i in range(n)]
    C = [Int(f'C_{k}') for k in range(m)]
    MaxCost = Int('MaxCost')

    # Constraints
    constraints = []

    # Domains for U
    for i in range(n):
        constraints.append(U[i] > 0)
        constraints.append(U[i] <= n)

    # C1: Each item must be delivered by exactly one courier
    for i in range(n):
        constraints.append(Or([X[k][i][j] for j in range(n + 1) if i != j for k in range(m)]))
        constraints.append(Or([X[k][j][i] for j in range(n + 1) if i != j for k in range(m)]))
        for k in range(m):
            constraints.append(at_most_one_T([X[k][j][i] for j in range(n + 1) if i != j]))

    # C2: Item assignment constraints
    for i in range(n):
        for k in range(m):
            constraints.append(Y[k][i] == Or([X[k][i][j] for j in range(n + 1) if i != j]))
            constraints.append(Y[k][i] == Or([X[k][j][i] for j in range(n + 1) if i != j]))
    for i in range(n):
        constraints.append(exactly_one_T([Y[k][i] for k in range(m)]))

    # C3: Load constraints
    for k in range(m):
        constraints.append(Sum([If(Y[k][i], s_values[i], 0) for i in range(n)]) <= l_values[k])

    # C4: Ensure couriers start and end at the origin
    for k in range(m):
        constraints.append(exactly_one_T([X[k][n][j] for j in range(n)]))
        constraints.append(exactly_one_T([X[k][i][n] for i in range(n)]))
        constraints.append(Or([Y[k][i] for i in range(n)]))

    # C5: Subtour elimination constraints
    for i in range(n):
        for j in range(n):
            if i != j:
                arc_traversed = Or([X[k][i][j] for k in range(m)])
                constraints.append(Implies(arc_traversed, U[j] > U[i]))

    # Cost constraints
    for k in range(m):
        constraints.append(C[k] == Sum([If(X[k][i][j], D[i][j], 0) for i in range(n + 1) for j in range(n + 1) if i != j]))
        constraints.append(MaxCost >= C[k])

    constraints.append(Or([MaxCost == C[k] for k in range(m)]))
    constraints.append(MaxCost <= UB)
    constraints.append(MaxCost >= LB)

    # Objective: Minimize the maximum distance traveled by any courier
    solver = Solver()
    solver.add(constraints)

    start_time = time.time()
    best_solution = None
    best_max_cost = UB

    # Incremental solving with smaller time chunks
    while UB > LB and time.time() - start_time < time_limit:
        mid = (UB + LB) // 2
        solver.push()
        solver.add(MaxCost <= mid)
        solver.set('timeout', 300 * 1000)  # Set timeout for the entire process
        if solver.check() == sat:
            model = solver.model()
            best_solution = {
                'max_distance': model.evaluate(MaxCost).as_long(),
                'assignments': [(k, i + 1) for k in range(m) for i in range(n) if is_true(model.evaluate(Y[k][i]))],
                'routes': [
                    [(i, j) for i in range(n + 1) for j in range(n + 1) if i != j and is_true(model.evaluate(X[k][i][j]))]
                    for k in range(m)
                ]
            }
            best_max_cost = model.evaluate(MaxCost).as_long()
            UB = best_max_cost
        else:
            LB = mid + 1
        solver.pop()

    elapsed_time = time.time() - start_time
    return best_solution, elapsed_time

In [41]:
def plot_routes(routes, instance, instance_name, method_name):
    n = instance['n']
    colors = ['r', 'g', 'b', 'c', 'm', 'y']

    coordinates = generate_coordinates(n)

    plt.figure(figsize=(8, 8))
    plt.grid(True)

    # Plot the nodes
    for i in range(n):
        plt.plot(*coordinates[i], 'o', color='white', markersize=10, markeredgecolor='black')
        plt.text(coordinates[i][0], coordinates[i][1], f'd{i+1}', fontsize=12, ha='right')

    # Plot the origin
    plt.plot(*coordinates[n], 'o', color='black', markersize=10)
    plt.text(coordinates[n][0], coordinates[n][1], 'origin', fontsize=12, ha='right')

    # Plot the routes
    for i, route in enumerate(routes):
        color = colors[i % len(colors)]
        for j in range(len(route) - 1):
            start = coordinates[route[j]]
            end = coordinates[route[j+1]]
            plt.arrow(start[0], start[1], end[0] - start[0], end[1] - start[1],
                      color=color, head_width=0.05, length_includes_head=True)

    plt.xlabel('x')
    plt.ylabel('y')

    # Create custom legend
    legend_elements = [Line2D([0], [0], color=colors[i % len(colors)], lw=2, label=f'Courier {i+1}')
                       for i in range(len(routes))]
    plt.legend(handles=legend_elements, loc='upper left')

    # Save plot
    if not os.path.exists('res/SMT/Solver_Binary/route_map'):
        os.makedirs('res/SMT/Solver_Binary/route_map')
    plt.savefig(f'res/SMT/Solver_Binary/route_map/{instance_name}_{method_name}_route.png')
    plt.close()

def save_route_json(instance_name, method_name, time_taken, optimal, obj, sol):
    if not os.path.exists('res/SMT/Solver_Binary/route'):
        os.makedirs('res/SMT/Solver_Binary/route')

    route_data = {
        method_name: {
            "time": time_taken,
            "optimal": optimal,
            "obj": obj,
            "sol": sol
        }
    }

    json_path = f'res/SMT/Solver_Binary/route/{instance_name}_route.json'
    if os.path.exists(json_path):
        with open(json_path, 'r') as f:
            data = json.load(f)
    else:
        data = {}

    data[method_name] = route_data[method_name]

    with open(json_path, 'w') as f:
        json.dump(data, f, indent=4)

In [42]:
folder_path = 'Instances'  
method_name = 'Solver_Binary'

def read_dat_files(folder_path):
    dat_files = [f for f in os.listdir(folder_path) if f.endswith('.dat')]
    instances = {}

    for dat_file in dat_files:
        file_path = os.path.join(folder_path, dat_file)
        instances[dat_file] = parseInstance(file_path)

    return instances

instances = read_dat_files(folder_path)

for instance_name, instance in instances.items():
    m = instance['m']
    n = instance['n']
    l_values = instance['l_values']
    s_values = instance['s_values']
    D = instance['distance_matrix']
    LB, UB = computeBounds(D, m, n)
    solution, time_taken = solve_mcp(m, n, l_values, s_values, D, LB, UB, time_limit=300)
    print_solution(instance_name, solution, instance, method_name, time_taken)


Instance: inst01.dat
Max Distance: 14
Courier 1: Items [1, 3, 4]
Courier 1 Route: [6, 3, 2, 0, 6]
Courier 2: Items [2, 5, 6]
Courier 2 Route: [6, 1, 4, 5, 6]
Instance: inst02.dat
Max Distance: 226
Courier 1: Items [3, 7]
Courier 1 Route: [9, 6, 2, 9]
Courier 2: Items [5, 9]
Courier 2 Route: [9, 4, 8, 9]
Courier 3: Items [2]
Courier 3 Route: [9, 1, 9]
Courier 4: Items [6, 8]
Courier 4 Route: [9, 7, 5, 9]
Courier 5: Items [1]
Courier 5 Route: [9, 0, 9]
Courier 6: Items [4]
Courier 6 Route: [9, 3, 9]
Instance: inst03.dat
Max Distance: 12
Courier 1: Items [2, 4, 5]
Courier 1 Route: [7, 4, 3, 1, 7]
Courier 2: Items [3, 6]
Courier 2 Route: [7, 2, 5, 7]
Courier 3: Items [1, 7]
Courier 3 Route: [7, 6, 0, 7]
Instance: inst04.dat
Max Distance: 220
Courier 1: Items [4]
Courier 1 Route: [10, 3, 10]
Courier 2: Items [6, 10]
Courier 2 Route: [10, 9, 5, 10]
Courier 3: Items [7, 9]
Courier 3 Route: [10, 6, 8, 10]
Courier 4: Items [2]
Courier 4 Route: [10, 1, 10]
Courier 5: Items [3]
Courier 5 Route: [

Solver_without_Binary

In [29]:
def solve_mcp(m, n, l_values, s_values, D, time_limit=300):
    # Decision variables
    X = [[[Bool(f'X_{k}_{i}_{j}') for j in range(n + 1)] for i in range(n + 1)] for k in range(m)]
    Y = [[Bool(f'Y_{k}_{i}') for i in range(n)] for k in range(m)]
    U = [Int(f'U_{i}') for i in range(n)]
    C = [Int(f'C_{k}') for k in range(m)]
    MaxCost = Int('MaxCost')

    # Constraints
    constraints = []

    # Domains for U
    for i in range(n):
        constraints.append(U[i] > 0)
        constraints.append(U[i] <= n)

    # C1: Each item must be delivered by exactly one courier
    for i in range(n):
        constraints.append(Or([X[k][i][j] for j in range(n + 1) if i != j for k in range(m)]))
        constraints.append(Or([X[k][j][i] for j in range(n + 1) if i != j for k in range(m)]))
        for k in range(m):
            constraints.append(at_most_one_T([X[k][j][i] for j in range(n + 1) if i != j]))

    # C2: Item assignment constraints
    for i in range(n):
        for k in range(m):
            constraints.append(Y[k][i] == Or([X[k][i][j] for j in range(n + 1) if i != j]))
            constraints.append(Y[k][i] == Or([X[k][j][i] for j in range(n + 1) if i != j]))
    for i in range(n):
        constraints.append(exactly_one_T([Y[k][i] for k in range(m)]))

    # C3: Load constraints
    for k in range(m):
        constraints.append(Sum([If(Y[k][i], s_values[i], 0) for i in range(n)]) <= l_values[k])

    # C4: Ensure couriers start and end at the origin
    for k in range(m):
        constraints.append(exactly_one_T([X[k][n][j] for j in range(n)]))
        constraints.append(exactly_one_T([X[k][i][n] for i in range(n)]))
        constraints.append(Or([Y[k][i] for i in range(n)]))

    # C5: Subtour elimination constraints
    for i in range(n):
        for j in range(n):
            if i != j:
                arc_traversed = Or([X[k][i][j] for k in range(m)])
                constraints.append(Implies(arc_traversed, U[j] > U[i]))

    # Cost constraints
    for k in range(m):
        constraints.append(C[k] == Sum([If(X[k][i][j], D[i][j], 0) for i in range(n + 1) for j in range(n + 1) if i != j]))
        constraints.append(MaxCost >= C[k])

    constraints.append(Or([MaxCost == C[k] for k in range(m)]))

    # Objective: Minimize the maximum distance traveled by any courier
    solver = Solver()
    solver.add(constraints)
    solver.set('timeout', time_limit * 1000)

    start_time = time.time()
    if solver.check() == sat:
        model = solver.model()
        best_solution = {
            'max_distance': model.evaluate(MaxCost).as_long(),
            'assignments': [(k, i + 1) for k in range(m) for i in range(n) if is_true(model.evaluate(Y[k][i]))],
            'routes': [
                [(i, j) for i in range(n + 1) for j in range(n + 1) if i != j and is_true(model.evaluate(X[k][i][j]))]
                for k in range(m)
            ]
        }
        elapsed_time = time.time() - start_time
        return best_solution, elapsed_time
    else:
        elapsed_time = time.time() - start_time
        return None, elapsed_time

In [32]:
def plot_routes(routes, instance, instance_name, method_name):
    n = instance['n']
    colors = ['r', 'g', 'b', 'c', 'm', 'y']

    coordinates = generate_coordinates(n)

    plt.figure(figsize=(8, 8))
    plt.grid(True)

    # Plot the nodes
    for i in range(n):
        plt.plot(*coordinates[i], 'o', color='white', markersize=10, markeredgecolor='black')
        plt.text(coordinates[i][0], coordinates[i][1], f'd{i+1}', fontsize=12, ha='right')

    # Plot the origin
    plt.plot(*coordinates[n], 'o', color='black', markersize=10)
    plt.text(coordinates[n][0], coordinates[n][1], 'origin', fontsize=12, ha='right')

    # Plot the routes
    for i, route in enumerate(routes):
        color = colors[i % len(colors)]
        for j in range(len(route) - 1):
            start = coordinates[route[j]]
            end = coordinates[route[j+1]]
            plt.arrow(start[0], start[1], end[0] - start[0], end[1] - start[1],
                      color=color, head_width=0.05, length_includes_head=True)

    plt.xlabel('x')
    plt.ylabel('y')

    # Create custom legend
    legend_elements = [Line2D([0], [0], color=colors[i % len(colors)], lw=2, label=f'Courier {i+1}')
                       for i in range(len(routes))]
    plt.legend(handles=legend_elements, loc='upper left')

    # Save plot
    if not os.path.exists('res/SMT/Solver_without_Binary/route_map'):
        os.makedirs('res/SMT/Solver_without_Binary/route_map')
    plt.savefig(f'res/SMT/Solver_without_Binary/route_map/{instance_name}_{method_name}_route.png')
    plt.close()

def save_route_json(instance_name, method_name, time_taken, optimal, obj, sol):
    if not os.path.exists('res/SMT/Solver_without_Binary/route'):
        os.makedirs('res/SMT/Solver_without_Binary/route')

    route_data = {
        method_name: {
            "time": time_taken,
            "optimal": optimal,
            "obj": obj,
            "sol": sol
        }
    }

    json_path = f'res/SMT/Solver_without_Binary/route/{instance_name}_route.json'
    if os.path.exists(json_path):
        with open(json_path, 'r') as f:
            data = json.load(f)
    else:
        data = {}

    data[method_name] = route_data[method_name]

    with open(json_path, 'w') as f:
        json.dump(data, f, indent=4)

In [33]:
folder_path = 'Instances'  
method_name = 'Solver_without_Binary'

def read_dat_files(folder_path):
    dat_files = [f for f in os.listdir(folder_path) if f.endswith('.dat')]
    instances = {}

    for dat_file in dat_files:
        file_path = os.path.join(folder_path, dat_file)
        instances[dat_file] = parseInstance(file_path)

    return instances

instances = read_dat_files(folder_path)

for instance_name, instance in instances.items():
    m = instance['m']
    n = instance['n']
    l_values = instance['l_values']
    s_values = instance['s_values']
    D = instance['distance_matrix']
    LB, UB = computeBounds(D, m, n)
    solution, time_taken = solve_mcp(m, n, l_values, s_values, D, time_limit=300)
    print_solution(instance_name, solution, instance, method_name, time_taken)


Instance: inst01.dat
Max Distance: 18
Courier 1: Items [3, 5, 6]
Courier 1 Route: [6, 4, 5, 2, 6]
Courier 2: Items [1, 2, 4]
Courier 2 Route: [6, 0, 1, 3, 6]
Instance: inst02.dat
Max Distance: 363
Courier 1: Items [3]
Courier 1 Route: [9, 2, 9]
Courier 2: Items [2, 6, 8]
Courier 2 Route: [9, 5, 7, 1, 9]
Courier 3: Items [9]
Courier 3 Route: [9, 8, 9]
Courier 4: Items [5, 7]
Courier 4 Route: [9, 6, 4, 9]
Courier 5: Items [4]
Courier 5 Route: [9, 3, 9]
Courier 6: Items [1]
Courier 6 Route: [9, 0, 9]
Instance: inst03.dat
Max Distance: 14
Courier 1: Items [2, 4, 5]
Courier 1 Route: [7, 1, 4, 3, 7]
Courier 2: Items [3, 7]
Courier 2 Route: [7, 2, 6, 7]
Courier 3: Items [1, 6]
Courier 3 Route: [7, 0, 5, 7]
Instance: inst04.dat
Max Distance: 220
Courier 1: Items [3]
Courier 1 Route: [10, 2, 10]
Courier 2: Items [9, 10]
Courier 2 Route: [10, 9, 8, 10]
Courier 3: Items [8]
Courier 3 Route: [10, 7, 10]
Courier 4: Items [2]
Courier 4 Route: [10, 1, 10]
Courier 5: Items [6]
Courier 5 Route: [10, 5,

OPTIMIZE WITHOUT SB

In [9]:
def solve_mcp(m, n, l_values, s_values, D, LB, UB, time_limit=300):
    # Decision variables
    X = [[[Bool(f'X_{k}_{i}_{j}') for j in range(n + 1)] for i in range(n + 1)] for k in range(m)]
    Y = [[Bool(f'Y_{k}_{i}') for i in range(n)] for k in range(m)]
    U = [Int(f'U_{i}') for i in range(n)]
    C = [Int(f'C_{k}') for k in range(m)]
    MaxCost = Int('MaxCost')

    # Constraints
    constraints = []

    # Domains for U
    for i in range(n):
        constraints.append(U[i] > 0)
        constraints.append(U[i] <= n)

    # C1: Each item must be delivered by exactly one courier
    for i in range(n):
        constraints.append(Or([X[k][i][j] for j in range(n + 1) if i != j for k in range(m)]))
        constraints.append(Or([X[k][j][i] for j in range(n + 1) if i != j for k in range(m)]))
        for k in range(m):
            constraints.append(at_most_one_T([X[k][j][i] for j in range(n + 1) if i != j]))

    # C2: Item assignment constraints
    for i in range(n):
        for k in range(m):
            constraints.append(Y[k][i] == Or([X[k][i][j] for j in range(n + 1) if i != j]))
            constraints.append(Y[k][i] == Or([X[k][j][i] for j in range(n + 1) if i != j]))
    for i in range(n):
        constraints.append(exactly_one_T([Y[k][i] for k in range(m)]))

    # C3: Load constraints
    for k in range(m):
        constraints.append(Sum([If(Y[k][i], s_values[i], 0) for i in range(n)]) <= l_values[k])

    # C4: Ensure couriers start and end at the origin
    for k in range(m):
        constraints.append(exactly_one_T([X[k][n][j] for j in range(n)]))
        constraints.append(exactly_one_T([X[k][i][n] for i in range(n)]))
        constraints.append(Or([Y[k][i] for i in range(n)]))

    # C5: Subtour elimination constraints
    for i in range(n):
        for j in range(n):
            if i != j:
                arc_traversed = Or([X[k][i][j] for k in range(m)])
                constraints.append(Implies(arc_traversed, U[j] > U[i]))

    # Cost constraints
    for k in range(m):
        constraints.append(C[k] == Sum([If(X[k][i][j], D[i][j], 0) for i in range(n + 1) for j in range(n + 1) if i != j]))
        constraints.append(MaxCost >= C[k])

    constraints.append(Or([MaxCost == C[k] for k in range(m)]))
    constraints.append(MaxCost <= UB)
    constraints.append(MaxCost >= LB)

    # Objective: Minimize the maximum distance traveled by any courier
    solver = Optimize()
    solver.add(constraints)
    solver.minimize(MaxCost)

    start_time = time.time()
    best_solution = None
    best_max_cost = UB

    # Incremental solving with smaller time chunks
    while UB > LB and time.time() - start_time < time_limit:
        mid = (UB + LB) // 2
        solver.push()
        solver.add(MaxCost <= mid)
        solver.set('timeout', int((time_limit - (time.time() - start_time)) * 1000))  # Set timeout for remaining time
        if solver.check() == sat:
            model = solver.model()
            best_solution = {
                'max_distance': model.evaluate(MaxCost).as_long(),
                'assignments': [(k, i + 1) for k in range(m) for i in range(n) if is_true(model.evaluate(Y[k][i]))],
                'routes': [
                    [(i, j) for i in range(n + 1) for j in range(n + 1) if i != j and is_true(model.evaluate(X[k][i][j]))]
                    for k in range(m)
                ]
            }
            best_max_cost = model.evaluate(MaxCost).as_long()
            UB = best_max_cost
        else:
            LB = mid + 1
        solver.pop()

    elapsed_time = time.time() - start_time
    return best_solution, elapsed_time

In [10]:
def plot_routes(routes, instance, instance_name, method_name):
    n = instance['n']
    colors = ['r', 'g', 'b', 'c', 'm', 'y']

    coordinates = generate_coordinates(n)

    plt.figure(figsize=(8, 8))
    plt.grid(True)

    # Plot the nodes
    for i in range(n):
        plt.plot(*coordinates[i], 'o', color='white', markersize=10, markeredgecolor='black')
        plt.text(coordinates[i][0], coordinates[i][1], f'd{i+1}', fontsize=12, ha='right')

    # Plot the origin
    plt.plot(*coordinates[n], 'o', color='black', markersize=10)
    plt.text(coordinates[n][0], coordinates[n][1], 'origin', fontsize=12, ha='right')

    # Plot the routes
    for i, route in enumerate(routes):
        color = colors[i % len(colors)]
        for j in range(len(route) - 1):
            start = coordinates[route[j]]
            end = coordinates[route[j+1]]
            plt.arrow(start[0], start[1], end[0] - start[0], end[1] - start[1],
                      color=color, head_width=0.05, length_includes_head=True)

    plt.xlabel('x')
    plt.ylabel('y')

    # Create custom legend
    legend_elements = [Line2D([0], [0], color=colors[i % len(colors)], lw=2, label=f'Courier {i+1}')
                       for i in range(len(routes))]
    plt.legend(handles=legend_elements, loc='upper left')

    # Save plot
    if not os.path.exists('/res/SMT/Optimizer/route_map'):
        os.makedirs('/res/SMT/Optimizer/route_map')
    plt.savefig(f'/res/SMT/Optimizer/route_map/{instance_name}_{method_name}_route.png')
    plt.close()

def save_route_json(instance_name, method_name, time_taken, optimal, obj, sol):
    if not os.path.exists('/res/SMT/Optimizer/route'):
        os.makedirs('/res/SMT/Optimizer/route')

    route_data = {
        method_name: {
            "time": time_taken,
            "optimal": optimal,
            "obj": obj,
            "sol": sol
        }
    }

    json_path = f'/res/SMT/Optimizer/route/{instance_name}_route.json'
    if os.path.exists(json_path):
        with open(json_path, 'r') as f:
            data = json.load(f)
    else:
        data = {}

    data[method_name] = route_data[method_name]

    with open(json_path, 'w') as f:
        json.dump(data, f, indent=4)

In [ ]:
folder_path = 'Instances'  
method_name = 'optimize'

def read_dat_files(folder_path):
    dat_files = [f for f in os.listdir(folder_path) if f.endswith('.dat')]
    instances = {}

    for dat_file in dat_files:
        file_path = os.path.join(folder_path, dat_file)
        instances[dat_file] = parseInstance(file_path)

    return instances

instances = read_dat_files(folder_path)

for instance_name, instance in instances.items():
    m = instance['m']
    n = instance['n']
    l_values = instance['l_values']
    s_values = instance['s_values']
    D = instance['distance_matrix']
    LB, UB = computeBounds(D, m, n)
    solution, time_taken = solve_mcp(m, n, l_values, s_values, D, LB, UB, time_limit=300)
    print_solution(instance_name, solution, instance, method_name, time_taken)

SMT_LIB+SB

In [28]:
# SMT Solver Helper Functions
counter = 0
unique_bool_declarations = []

def generate_unique_bool_name():
    global counter
    counter += 1
    name = f"unique_bool_{counter}"
    unique_bool_declarations.append(name)
    return name

def at_most_one_T(bools):
    if len(bools) <= 4:  # base case
        return And([Not(And(b1, b2)) for b1, b2 in combinations(bools, 2)])
    
    # recursive step
    y = Bool(generate_unique_bool_name())
    first = bools[:3]
    first.append(y)
    c_first = at_most_one_T(first)

    last = bools[3:]
    last.insert(0, Not(y))
    c_last = at_most_one_T(last)

    return And(c_first, c_last)

def exactly_one_T(bools):
    return And(at_most_one_T(bools), Or(bools))

In [29]:
def generate_smtlib(m, n, l_values, s_values, D, LB, UB):
    smtlib = []

    # Declare variables
    for k in range(m):
        for i in range(n + 1):
            for j in range(n + 1):
                if i != j:
                    smtlib.append(f"(declare-const X_{k}_{i}_{j} Bool)")

    for k in range(m):
        for i in range(n):
            smtlib.append(f"(declare-const Y_{k}_{i} Bool)")

    for i in range(n):
        smtlib.append(f"(declare-const U_{i} Int)")

    for k in range(m):
        smtlib.append(f"(declare-const C_{k} Int)")

    smtlib.append(f"(declare-const MaxCost Int)")

    # Declare all unique bool vars
    for var in all_bool_vars:
        smtlib.append(f"(declare-const {var} Bool)")

    # Constraints
    for i in range(n):
        smtlib.append(f"(assert (> U_{i} 0))")
        smtlib.append(f"(assert (<= U_{i} {n}))")

    for i in range(n):
        or_clause = " ".join([f"X_{k}_{i}_{j}" for j in range(n + 1) if i != j for k in range(m)])
        smtlib.append(f"(assert (or {or_clause}))")

        or_clause = " ".join([f"X_{k}_{j}_{i}" for j in range(n + 1) if i != j for k in range(m)])
        smtlib.append(f"(assert (or {or_clause}))")

        for k in range(m):
            at_most_one = at_most_one_T([Bool(f"X_{k}_{j}_{i}") for j in range(n + 1) if i != j])
            smtlib.append(f"(assert {at_most_one.sexpr()})")

    for i in range(n):
        for k in range(m):
            y_clause = " ".join([f"X_{k}_{i}_{j}" for j in range(n + 1) if i != j])
            smtlib.append(f"(assert (= Y_{k}_{i} (or {y_clause})))")
            y_clause = " ".join([f"X_{k}_{j}_{i}" for j in range(n + 1) if i != j])
            smtlib.append(f"(assert (= Y_{k}_{i} (or {y_clause})))")

    for i in range(n):
        exactly_one = exactly_one_T([Bool(f"Y_{k}_{i}") for k in range(m)])
        smtlib.append(f"(assert {exactly_one.sexpr()})")

    for k in range(m):
        load_constraint = " ".join([f"(ite Y_{k}_{i} {s_values[i]} 0)" for i in range(n)])
        smtlib.append(f"(assert (<= (+ {load_constraint}) {l_values[k]}))")

    for k in range(m):
        exactly_one_start = exactly_one_T([Bool(f"X_{k}_{n}_{j}") for j in range(n)])
        smtlib.append(f"(assert {exactly_one_start.sexpr()})")

        exactly_one_end = exactly_one_T([Bool(f"X_{k}_{i}_{n}") for i in range(n)])
        smtlib.append(f"(assert {exactly_one_end.sexpr()})")

        smtlib.append(f"(assert (or {' '.join([f'Y_{k}_{i}' for i in range(n)])}))")

    for i in range(n):
        for j in range(n):
            if i != j:
                arc_traversed = " ".join([f"X_{k}_{i}_{j}" for k in range(m)])
                smtlib.append(f"(assert (=> (or {arc_traversed}) (> U_{j} U_{i})))")

    for k in range(m):
        cost_constraint = " ".join([f"(ite X_{k}_{i}_{j} {D[i][j]} 0)" for i in range(n + 1) for j in range(n + 1) if i != j])
        smtlib.append(f"(assert (= C_{k} (+ {cost_constraint})))")
        smtlib.append(f"(assert (>= MaxCost C_{k}))")

    smtlib.append(f"(assert (or {' '.join([f'(= MaxCost C_{k})' for k in range(m)])}))")
    smtlib.append(f"(assert (<= MaxCost {UB}))")
    smtlib.append(f"(assert (>= MaxCost {LB}))")

    # Objective function
    smtlib.append(f"(minimize MaxCost)")

    return "\n".join(smtlib)

def solve_smtlib(smtlib, time_limit):
    solver = Optimize()
    solver.from_string(smtlib)
    solver.set('timeout', time_limit * 1000)

    start_time = time.time()
    if solver.check() == sat:
        model = solver.model()
        best_solution = {
            'max_distance': model.evaluate(Int('MaxCost')).as_long(),
            'assignments': [(k, i + 1) for k in range(m) for i in range(n) if is_true(model.evaluate(Bool(f"Y_{k}_{i}")))],
            'routes': [
                [(i, j) for i in range(n + 1) for j in range(n + 1) if i != j and is_true(model.evaluate(Bool(f"X_{k}_{i}_{j}")))]
                for k in range(m)
            ]
        }
        elapsed_time = time.time() - start_time
        return best_solution, elapsed_time
    else:
        return None, time.time() - start_time

In [30]:
def plot_routes(routes, instance, instance_name, method_name):
    n = instance['n']
    colors = ['r', 'g', 'b', 'c', 'm', 'y']

    coordinates = generate_coordinates(n)

    plt.figure(figsize=(8, 8))
    plt.grid(True)

    # Plot the nodes
    for i in range(n):
        plt.plot(*coordinates[i], 'o', color='white', markersize=10, markeredgecolor='black')
        plt.text(coordinates[i][0], coordinates[i][1], f'd{i+1}', fontsize=12, ha='right')

    # Plot the origin
    plt.plot(*coordinates[n], 'o', color='black', markersize=10)
    plt.text(coordinates[n][0], coordinates[n][1], 'origin', fontsize=12, ha='right')

    # Plot the routes
    for i, route in enumerate(routes):
        color = colors[i % len(colors)]
        for j in range(len(route) - 1):
            start = coordinates[route[j]]
            end = coordinates[route[j+1]]
            plt.arrow(start[0], start[1], end[0] - start[0], end[1] - start[1],
                      color=color, head_width=0.05, length_includes_head=True)

    plt.xlabel('x')
    plt.ylabel('y')

    # Create custom legend
    legend_elements = [Line2D([0], [0], color=colors[i % len(colors)], lw=2, label=f'Courier {i+1}')
                       for i in range(len(routes))]
    plt.legend(handles=legend_elements, loc='upper left')

    # Save plot
    if not os.path.exists('/res/SMT/SMT_LIB_SB/route_map'):
        os.makedirs('/res/SMT/SMT_LIB_SB/route_map')
    plt.savefig(f'/res/SMT/SMT_LIB_SB/route_map/{instance_name}_{method_name}_route.png')
    plt.close()

def save_route_json(instance_name, method_name, time_taken, optimal, obj, sol):
    if not os.path.exists('/res/SMT/SMT_LIB_SB/route'):
        os.makedirs('/res/SMT/SMT_LIB_SB/route')

    route_data = {
        method_name: {
            "time": time_taken,
            "optimal": optimal,
            "obj": obj,
            "sol": sol
        }
    }

    json_path = f'/res/SMT/SMT_LIB_SB/route/{instance_name}_route.json'
    if os.path.exists(json_path):
        with open(json_path, 'r') as f:
            data = json.load(f)
    else:
        data = {}

    data[method_name] = route_data[method_name]

    with open(json_path, 'w') as f:
        json.dump(data, f, indent=4)

In [ ]:
folder_path = 'Instances'  # Adjust this path as necessary
method_name = 'smtlib+sb'  # Example method name

def read_dat_files(folder_path):
    dat_files = [f for f in os.listdir(folder_path) if f.endswith('.dat')]
    instances = {}

    for dat_file in dat_files:
        file_path = os.path.join(folder_path, dat_file)
        instances[dat_file] = parseInstance(file_path)

    return instances

instances = read_dat_files(folder_path)

for instance_name, instance in instances.items():
    m = instance['m']
    n = instance['n']
    l_values = instance['l_values']
    s_values = instance['s_values']
    D = instance['distance_matrix']
    LB, UB = computeBounds(D, m, n)
    smtlib = generate_smtlib(m, n, l_values, s_values, D, LB, UB)
    solution, time_taken = solve_smtlib(smtlib, time_limit=300)
    print_solution(instance_name, solution, instance, method_name, time_taken)

SMTLIB

In [5]:
def generate_smtlib(m, n, l_values, s_values, D, LB, UB):
    smtlib = []

    # Declare variables
    for k in range(m):
        for i in range(n + 1):
            for j in range(n + 1):
                if i != j:
                    smtlib.append(f"(declare-const X_{k}_{i}_{j} Bool)")

    for k in range(m):
        for i in range(n):
            smtlib.append(f"(declare-const Y_{k}_{i} Bool)")

    for i in range(n):
        smtlib.append(f"(declare-const U_{i} Int)")

    for k in range(m):
        smtlib.append(f"(declare-const C_{k} Int)")

    smtlib.append(f"(declare-const MaxCost Int)")

    # Add constraints
    for i in range(n):
        smtlib.append(f"(assert (> U_{i} 0))")
        smtlib.append(f"(assert (<= U_{i} {n}))")

    for i in range(n):
        or_clause = " ".join([f"X_{k}_{i}_{j}" for j in range(n + 1) if i != j for k in range(m)])
        smtlib.append(f"(assert (or {or_clause}))")

        or_clause = " ".join([f"X_{k}_{j}_{i}" for j in range(n + 1) if i != j for k in range(m)])
        smtlib.append(f"(assert (or {or_clause}))")

        for k in range(m):
            at_most_one = at_most_one_T([Bool(f"X_{k}_{j}_{i}") for j in range(n + 1) if i != j])
            smtlib.append(f"(assert {at_most_one.sexpr()})")

    for i in range(n):
        for k in range(m):
            y_clause = " ".join([f"X_{k}_{i}_{j}" for j in range(n + 1) if i != j])
            smtlib.append(f"(assert (= Y_{k}_{i} (or {y_clause})))")
            y_clause = " ".join([f"X_{k}_{j}_{i}" for j in range(n + 1) if i != j])
            smtlib.append(f"(assert (= Y_{k}_{i} (or {y_clause})))")

    for i in range(n):
        exactly_one = exactly_one_T([Bool(f"Y_{k}_{i}") for k in range(m)])
        smtlib.append(f"(assert {exactly_one.sexpr()})")

    for k in range(m):
        load_constraint = " ".join([f"(ite Y_{k}_{i} {s_values[i]} 0)" for i in range(n)])
        smtlib.append(f"(assert (<= (+ {load_constraint}) {l_values[k]}))")

    for k in range(m):
        exactly_one_start = exactly_one_T([Bool(f"X_{k}_{n}_{j}") for j in range(n)])
        smtlib.append(f"(assert {exactly_one_start.sexpr()})")

        exactly_one_end = exactly_one_T([Bool(f"X_{k}_{i}_{n}") for i in range(n)])
        smtlib.append(f"(assert {exactly_one_end.sexpr()})")

        smtlib.append(f"(assert (or {' '.join([f'Y_{k}_{i}' for i in range(n)])}))")

    for i in range(n):
        for j in range(n):
            if i != j:
                arc_traversed = " ".join([f"X_{k}_{i}_{j}" for k in range(m)])
                smtlib.append(f"(assert (=> (or {arc_traversed}) (> U_{j} U_{i})))")

    for k in range(m):
        cost_constraint = " ".join([f"(ite X_{k}_{i}_{j} {D[i][j]} 0)" for i in range(n + 1) for j in range(n + 1) if i != j])
        smtlib.append(f"(assert (= C_{k} (+ {cost_constraint})))")
        smtlib.append(f"(assert (>= MaxCost C_{k}))")

    smtlib.append(f"(assert (or {' '.join([f'(= MaxCost C_{k})' for k in range(m)])}))")
    smtlib.append(f"(assert (<= MaxCost {UB}))")
    smtlib.append(f"(assert (>= MaxCost {LB}))")

    # Objective function
    smtlib.append(f"(minimize MaxCost)")

    return "\n".join(smtlib)

def solve_smtlib(smtlib, time_limit):
    solver = Optimize()
    solver.from_string(smtlib)
    solver.set('timeout', time_limit * 1000)

    start_time = time.time()
    if solver.check() == sat:
        model = solver.model()
        best_solution = {
            'max_distance': model.evaluate(Int('MaxCost')).as_long(),
            'assignments': [(k, i + 1) for k in range(m) for i in range(n) if is_true(model.evaluate(Bool(f"Y_{k}_{i}")))],
            'routes': [
                [(i, j) for i in range(n + 1) for j in range(n + 1) if i != j and is_true(model.evaluate(Bool(f"X_{k}_{i}_{j}")))]
                for k in range(m)
            ]
        }
        elapsed_time = time.time() - start_time
        return best_solution, elapsed_time
    else:
        return None, time.time() - start_time

In [6]:
def plot_routes(routes, instance, instance_name, method_name):
    n = instance['n']
    colors = ['r', 'g', 'b', 'c', 'm', 'y']

    coordinates = generate_coordinates(n)

    plt.figure(figsize=(8, 8))
    plt.grid(True)

    # Plot the nodes
    for i in range(n):
        plt.plot(*coordinates[i], 'o', color='white', markersize=10, markeredgecolor='black')
        plt.text(coordinates[i][0], coordinates[i][1], f'd{i+1}', fontsize=12, ha='right')

    # Plot the origin
    plt.plot(*coordinates[n], 'o', color='black', markersize=10)
    plt.text(coordinates[n][0], coordinates[n][1], 'origin', fontsize=12, ha='right')

    # Plot the routes
    for i, route in enumerate(routes):
        color = colors[i % len(colors)]
        for j in range(len(route) - 1):
            start = coordinates[route[j]]
            end = coordinates[route[j+1]]
            plt.arrow(start[0], start[1], end[0] - start[0], end[1] - start[1],
                      color=color, head_width=0.05, length_includes_head=True)

    plt.xlabel('x')
    plt.ylabel('y')

    # Create custom legend
    legend_elements = [Line2D([0], [0], color=colors[i % len(colors)], lw=2, label=f'Courier {i+1}')
                       for i in range(len(routes))]
    plt.legend(handles=legend_elements, loc='upper left')

    # Save plot
    if not os.path.exists('/res/SMT/SMT_LIB/route_map'):
        os.makedirs('/res/SMT/SMT_LIB/route_map')
    plt.savefig(f'/res/SMT/SMT_LIB/route_map/{instance_name}_{method_name}_route.png')
    plt.close()

def save_route_json(instance_name, method_name, time_taken, optimal, obj, sol):
    if not os.path.exists('/res/SMT/SMT_LIB/route'):
        os.makedirs('/res/SMT/SMT_LIB/route')

    route_data = {
        method_name: {
            "time": time_taken,
            "optimal": optimal,
            "obj": obj,
            "sol": sol
        }
    }

    json_path = f'/res/SMT/SMT_LIB/route/{instance_name}_route.json'
    if os.path.exists(json_path):
        with open(json_path, 'r') as f:
            data = json.load(f)
    else:
        data = {}

    data[method_name] = route_data[method_name]

    with open(json_path, 'w') as f:
        json.dump(data, f, indent=4)

In [7]:
folder_path = 'Instances'  
method_name = 'SMT_LIB'

def read_dat_files(folder_path):
    dat_files = [f for f in os.listdir(folder_path) if f.endswith('.dat')]
    instances = {}

    for dat_file in dat_files:
        file_path = os.path.join(folder_path, dat_file)
        instances[dat_file] = parseInstance(file_path)

    return instances

instances = read_dat_files(folder_path)

for instance_name, instance in instances.items():
    m = instance['m']
    n = instance['n']
    l_values = instance['l_values']
    s_values = instance['s_values']
    D = instance['distance_matrix']
    LB, UB = computeBounds(D, m, n)
    solution, time_taken = solve_mcp(m, n, l_values, s_values, D, LB, UB, time_limit=300)
    print_solution(instance_name, solution, instance, method_name, time_taken)

NameError: name 'solve_mcp' is not defined